# Smoke Sensory Python Script
This script is designed to streamline the data acquisition and mining operation for LAFFORT. The source code herein was made in collaboration with Daniel A. Dycus, Technical Manager for LAFFORTUSA. The purpose of this script is outlined in the following sections of this Jupyter notebook.

* Import the csv file from Pigeonhole. (future instances for application development should use an API)
* Build a static dataframe.
* Use a series of nested loops to import data into static dataframe.
* Output new dataframe for data analysis.
* Join all frames for master sensory data file for each sensory session i.e. Each sensory session should possess a new dashboard.
* Export frame to xls format for current developmental needs.
* Future mission should acquire frame, perform analysis, and present dashboard. (6 month project?)

## Pigeonhole   

Pigeonhole reports come as xls or pdf. We currently save the files as csv and have not considered scraping the pdf. This should be done in batches in the future. For now, we focus on building a nest which will work for each xls we receive. We typically receive around fourty csv files in each batch. For data analysis, each csv represents all the sensory scores for a particular factor or treatment. In this experiment we capture sensory data from humans who taste the addition of different wine additives on wines made from smoke-exposed grapes. 

The csv has a header with the treatment name. First, we acquire the treatment label and will use it as our filename later. 


In [ ]:
# Put the pre-requisite libraries here
import pandas as pd 
from google.colab import files
uploaded = files.upload()

Saving CabernetFrancCaseiPlus50ghL.pdf to CabernetFrancCaseiPlus50ghL.pdf
Saving CabernetFranc-CaseiPlus50ghL.xlsx to CabernetFranc-CaseiPlus50ghL.xlsx
Saving CabernetFrancCaseiPlus100ghL.pdf to CabernetFrancCaseiPlus100ghL.pdf
Saving CabernetFranc-CaseiPlus100ghL.xlsx to CabernetFranc-CaseiPlus100ghL.xlsx
Saving CabernetFrancControl.pdf to CabernetFrancControl.pdf
Saving CabernetFranc-Control.xlsx to CabernetFranc-Control.xlsx
Saving CabernetFrancGeosorb50ghL.pdf to CabernetFrancGeosorb50ghL.pdf
Saving CabernetFranc-Geosorb50ghL.xlsx to CabernetFranc-Geosorb50ghL.xlsx
Saving CabernetFrancGeosorb100ghL.pdf to CabernetFrancGeosorb100ghL.pdf
Saving CabernetFranc-Geosorb100ghL.xlsx to CabernetFranc-Geosorb100ghL.xlsx
Saving CabernetFrancNOBILEAmericanFresh5gL.pdf to CabernetFrancNOBILEAmericanFresh5gL.pdf
Saving CabernetFrancNOBILE-AmericanFresh5gL.xlsx to CabernetFrancNOBILE-AmericanFresh5gL.xlsx
Saving CabernetFrancNOBILEAmericanFresh10ghL.pdf to CabernetFrancNOBILEAmericanFresh10ghL.pd

In [ ]:
import glob
survey_files = glob.glob('*.xlsx')
survey_files

['PinotNoir-YeastDerivativeA100ghL.xlsx',
 'Merlot-Control.xlsx',
 'PinotNoir-YeastDerivativeB50ghL.xlsx',
 'PracticeSession.xlsx',
 'CommercialViabilityofControls.xlsx',
 'CabernetFrancNOBILE-P75gL.xlsx',
 'PinotNoir-Geosorb60ghL.xlsx',
 'Merlot-Geosorb40ghL.xlsx',
 'CSNOBILE-FreshGran24mo5gL.xlsx',
 'CabernetFranc-Geosorb100ghL.xlsx',
 'ShortQuiz.xlsx',
 'CabernetFranc-Polylact50ghL.xlsx',
 'CabernetFranc-CaseiPlus50ghL.xlsx',
 'CabernetFrancNOBILE-AmericanFresh10ghL.xlsx',
 'CS-Geosorb50ghL.xlsx',
 'CS-Metschnikowia100ghL.xlsx',
 'CS-Metschnikowia10ghL.xlsx',
 'OverallProductPerformance.xlsx',
 'SmokeCharacterCloud.xlsx',
 'CSNOBILE-FreshGran24mo10gL.xlsx',
 'CabernetFrancNOBILE-FreshGran24mo5gL.xlsx',
 'PinotNoir-Geosorb40ghL.xlsx',
 'CS-SkimMilk2mLL.xlsx',
 'CabernetFrancNOBILE-P710gL.xlsx',
 'CabernetFrancNOBILE-FreshGran24mo10gL.xlsx',
 'FeedbackSurvey.xlsx',
 'PinotNoir-YeastDerivativeA50ghL.xlsx',
 'CSNOBILE-SweetVanilla10gL.xlsx',
 'CSNOBILE-OriginalSpice5gL.xlsx',
 'Cabernet

In [ ]:
file_name = "CabernetFranc-CaseiPlus50ghL.xlsx"
def get_raw_dataframe(file_name):
    import os
    survey_name = os.path.splitext(file_name)[0]
    xl_file = pd.ExcelFile(file_name)
    data_raw = {sheet_name: xl_file.parse(sheet_name) 
            for sheet_name in xl_file.sheet_names}
    df_raw = data_raw['Poll By Users']
    return survey_name, df_raw

survey_name, df_raw = get_raw_dataframe(file_name)
df_raw

,Pigeonhole Live Poll,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96
0,325565: Laffort Internal Smoke Sensory Analysi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1539862: Cabernet Franc Casei Plus 50g/hL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Voters' Attendee Profiles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Poll Votes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date/Time,Attendee Code,Name,Email,Year Born,Years of Tasting Experience,Ethnicity,Country,Wine Region you work in,Gender,Q1: 2,Q1: 3,Q1: 4,Q1: 5,Q1: 6,Q1: 7,Q2: 1,Q2: 2,Q2: 3,Q2: 4,Q2: 5,Q2: 6,Q3: 1,Q3: 3,Q3: 6,Q4: 1,Q4: 2,Q4: 3,Q4: 4,Q4: 5,Q5: 1,Q5: 2,Q5: 3,Q5: 4,Q5: 5,Q6: 1,Q6: 2,Q6: 3,Q6: 4,Q6: 5,...,Q9: 4,Q9: 5,Q10: 1,Q10: 2,Q10: 3,Q10: 4,Q10: 5,Q11: 1,Q11: 3,Q11: 4,Q11: 5,Q11: 6,Q11: 7,Q11: 8,Q12: 1,Q12: 2,Q12: 3,Q12: 4,Q12: 5,Q12: 7,Q13: 1,Q13: 2,Q13: 3,Q13: 4,Q13: 5,Q14: 1,Q14: 2,Q14: 3,Q14: 4,Q14: 5,Q14: 6,Q14: 7,Q14: 8,Q15: 1,Q15: 2,Q15: 3,Q15: 4,Q15: 6,Q15: 9,NaN
5,2021-01-21 12:25:35,FANCYPIGEON,Shaun Richardson,shaun.richardson@laffort.com,1968,30,Caucasian,USA,Sonoma,Male,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,...,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN
6,2021-01-21 13:30:14,SPINIFEXPIGEON,Cass Gossage,cass.gossage@laffort.com,1979,14,caucasian,United States,Sonoma/Napa,male,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,...,NaN,✓,NaN,NaN,NaN,✓,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,✓,NaN,NaN,NaN,NaN
7,2021-01-21 15:12:16,JACOBINPIGEON,Daniel Dycus,daniel.dycus@laffort.com,1982,15,White,USA,USA,M,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,✓,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,✓,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Dataframe
Our dataframe will have the following sensory attributes rated by each panelist. 


In [ ]:
def initial_cleaning(df_raw, rows_to_skip=4, cols_for_user_info=10):
    df=df_raw.drop(range(rows_to_skip))
    #create new header
    new_header = df.iloc[0] #grab the first row for the header
    df=df[1:] #take the data less the header row
    df.columns=new_header #set the header row as the df header
    #Slice the stuff I don't need to iterate over
    df_users_info=df.iloc[:,:cols_for_user_info]
    df_scores=df.iloc[:,cols_for_user_info:]
    df_scores = df_scores == '✓' # print('\u2713')
    return df_users_info, df_scores

df_users_info, df_scores = initial_cleaning(df_raw)
df_scores

4,Q1: 2,Q1: 3,Q1: 4,Q1: 5,Q1: 6,Q1: 7,Q2: 1,Q2: 2,Q2: 3,Q2: 4,Q2: 5,Q2: 6,Q3: 1,Q3: 3,Q3: 6,Q4: 1,Q4: 2,Q4: 3,Q4: 4,Q4: 5,Q5: 1,Q5: 2,Q5: 3,Q5: 4,Q5: 5,Q6: 1,Q6: 2,Q6: 3,Q6: 4,Q6: 5,Q6: 6,Q6: 7,Q7: 1,Q7: 2,Q7: 3,Q7: 4,Q7: 5,Q7: 7,Q8: 1,Q8: 2,...,Q9: 4,Q9: 5,Q10: 1,Q10: 2,Q10: 3,Q10: 4,Q10: 5,Q11: 1,Q11: 3,Q11: 4,Q11: 5,Q11: 6,Q11: 7,Q11: 8,Q12: 1,Q12: 2,Q12: 3,Q12: 4,Q12: 5,Q12: 7,Q13: 1,Q13: 2,Q13: 3,Q13: 4,Q13: 5,Q14: 1,Q14: 2,Q14: 3,Q14: 4,Q14: 5,Q14: 6,Q14: 7,Q14: 8,Q15: 1,Q15: 2,Q15: 3,Q15: 4,Q15: 6,Q15: 9,NaN
5,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
6,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False
7,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
8,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False
9,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
10,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
11,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,True,...,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
12,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,

In [ ]:
def pivot_scores(df_scores, survey_name, total_questions=15, rename_col_names=True):
    df_cols = []
    for question_num in range(1,total_questions+1):
        df_cols.append(f'Q{question_num}')
    
    df_pivot=pd.DataFrame(columns=df_cols, index=df_scores.index.values)
    for question_num in range(1,16):
        for score in range(10):
            if f'Q{question_num}: {score}' in df_scores.columns:
                for row in df_scores.index.values:
                    if df_scores[f'Q{question_num}: {score}'][row]==True:
                        # print(df[f'Q{q}: {score}'][row])
                        # print(f'Q{q}: {score}')
                        df_pivot[f'Q{question_num}'][row] = score

    df_cleaned = pd.concat([df_users_info, df_pivot], axis=1)
    df_cleaned["survey_name"] = survey_name
    df_cleaned["wine_type"] = survey_name.split("-")[0]
    df_cleaned["treatment"] = survey_name.split("-")[1]
    survey_name_df = df_cleaned.pop("survey_name")
    df_cleaned.insert(0, "survey_name", survey_name_df)

    if rename_col_names:
        questions=['Aroma|Fruity','Aroma|Herbaceous','Aroma|Smoky','Aroma|Ashy','Aroma|Tar', 
                'Flavor|Fruity', 'Flavor|Herbaceous','Flavor|Smoky','Flavor|Ashy', 'Flavor|Tar', 
                'Mouthfeel|Astringent','Mouthfeel|Bitter','Mouthfeel|Round','Mouthfeel|Lingering Ash','Mouthfeel|Metallic']
        i = 0
        for question_num in range(1,total_questions+1):
            df_cleaned = df_cleaned.rename(columns={f'Q{question_num}':questions[i]})
            i += 1
        
    return df_pivot, df_cleaned

df_pivot, df_cleaned = pivot_scores(df_scores, survey_name)
df_cleaned

,survey_name,Date/Time,Attendee Code,Name,Email,Year Born,Years of Tasting Experience,Ethnicity,Country,Wine Region you work in,Gender,Aroma|Fruity,Aroma|Herbaceous,Aroma|Smoky,Aroma|Ashy,Aroma|Tar,Flavor|Fruity,Flavor|Herbaceous,Flavor|Smoky,Flavor|Ashy,Flavor|Tar,Mouthfeel|Astringent,Mouthfeel|Bitter,Mouthfeel|Round,Mouthfeel|Lingering Ash,Mouthfeel|Metallic,wine_type,treatment
5,CSNOBILE-Base10gL,2021-01-22 14:34:09,ICEPIGEON,Jillian Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,5,5,NaN,NaN,5,3,4,2,NaN,4,3,6,2,2,CSNOBILE,Base10gL
6,CSNOBILE-Base10gL,2021-01-22 14:40:35,RACINGPIGEON,Garett Savage,garett.savage@laffort.com,1978,22,Asian American,USA,NaN,NaN,7,4,3,2,1,7,4,3,1,1,6,1,8,2,1,CSNOBILE,Base10gL
7,CSNOBILE-Base10gL,2021-01-22 14:41:04,CARRIERPIGEON,Carrier Pigeon,NaN,1986,10,Caucasian,Better Island,NaN,NaN,5,3,2,3,3,5,2,1,3,1,2,3,5,7,5,CSNOBILE,Base10gL
8,CSNOBILE-Base10gL,2021-01-22 16:36:17,FANCYPIGEON,Shaun Richardson,shaun.richardson@laffort.com,1968,30,Caucasian,USA,Sonoma,Male,4,6,5,7,5,5,6,5,7,5,5,3,4,6,5,CSNOBILE,Base10gL
9,CSNOBILE-Base10gL,2021-01-22 18:54:16,JACOBINPIGEON,Daniel Dycus,daniel.dycus@laffort.com,1982,15,White,USA,USA,M,5,3,8,7,8,4,2,9,NaN,NaN,5,8,3,NaN,9,CSNOBILE,Base10gL
10,CSNOBILE-Base10gL,2021-01-24 13:53:11,ARCHANGELPIGEON,Lindsey Otis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1,1,1,1,5,1,1,1,1,5,1,2,1,1,CSNOBILE,Base10gL
11,CSNOBILE-Base10gL,2021-01-24 15:42:03,BRONZEWINGPIGEON,Kelly Gifford,kelly.gifford@laffort.com,1988,NaN,Kennewick,United States,Pacific Northwest,Female,8,1,1,1,1,7,3,1,1,1,5,3,5,1,1,CSNOBILE,Base10gL
12,CSNOBILE-Base10gL,2021-01-25 14:19:15,HOMINGPIGEON,Matt Ridge,matt.ridge@laffort.com,1972,23,Irish America,USA,Western US,Male,4,5,1,1,1,3,5,1,1,1,1,1,5,1,1,CSNOBILE,Base10gL
13,CSNOBILE-Base10gL,2021-01-25 20:18:03,NICOBARPIGEON,Marcy Mallette,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,2,1,1,3,5,1,1,1,2,1,6,1,1,CSNOBILE,Base10gL


## Main Function
To process all the files in the directory and clean them all and create a "MASTER DATASET".

In [ ]:
survey_master_dataset = pd.DataFrame()
list_of_num_participants = []
list_of_num_questions = []
surveys_meta = pd.DataFrame(columns=["survey_name", "wine_type", "treatment", "num_participants", "num_questions"])

for file_name in survey_files:
    print(file_name)
    if file_name in ["OverallProductPerformance.xlsx", "SmokeCharacterCloud.xlsx", "FeedbackSurvey.xlsx", "CommercialViabilityofControls.xlsx", "PracticeSession.xlsx", "ShortQuiz.xlsx"]:
        print("PASSED THIS FILE ------------")
        continue
    survey_name, df_raw = get_raw_dataframe(file_name=file_name)
    df_users_info, df_scores = initial_cleaning(df_raw)
    df_pivot, df_cleaned = pivot_scores(df_scores, survey_name)
    survey_master_dataset = pd.concat([survey_master_dataset, df_cleaned])
    
    # below we wanna know more about what each file contains: how many participants and how many questions in each survey
    num_participants = df_pivot.shape[0]
    num_questions = df_pivot.shape[1]
    print(f": number_of_participants={num_participants}, " + f": number_of_questions={num_questions}")
    if num_participants not in list_of_num_participants:    
        list_of_num_participants.append(num_participants)
    if num_questions not in list_of_num_questions:    
        list_of_num_questions.append(num_questions)
    
    survey_info = pd.DataFrame(data={"survey_name": [survey_name], "wine_type": [survey_name.split("-")[0]], "treatment": [survey_name.split("-")[1]], 
                                     "num_participants": [num_participants], "num_questions": [num_questions]})
    surveys_meta = pd.concat([surveys_meta, survey_info]).reset_index(drop=True)

    


PinotNoir-YeastDerivativeA100ghL.xlsx
: number_of_participants=16, : number_of_questions=15
Merlot-Control.xlsx
: number_of_participants=11, : number_of_questions=15
PinotNoir-YeastDerivativeB50ghL.xlsx
: number_of_participants=16, : number_of_questions=15
PracticeSession.xlsx
PASSED THIS FILE ------------
CommercialViabilityofControls.xlsx
PASSED THIS FILE ------------
CabernetFrancNOBILE-P75gL.xlsx
: number_of_participants=11, : number_of_questions=15
PinotNoir-Geosorb60ghL.xlsx
: number_of_participants=16, : number_of_questions=15
Merlot-Geosorb40ghL.xlsx
: number_of_participants=11, : number_of_questions=15
CSNOBILE-FreshGran24mo5gL.xlsx
: number_of_participants=9, : number_of_questions=15
CabernetFranc-Geosorb100ghL.xlsx
: number_of_participants=11, : number_of_questions=15
ShortQuiz.xlsx
PASSED THIS FILE ------------
CabernetFranc-Polylact50ghL.xlsx
: number_of_participants=11, : number_of_questions=15
CabernetFranc-CaseiPlus50ghL.xlsx
: number_of_participants=11, : number_of_que

In [ ]:
list_of_num_questions

[15]

In [ ]:
list_of_num_participants

[16, 11, 9, 10]

In [ ]:
surveys_meta

,survey_name,wine_type,treatment,num_participants,num_questions
0,PinotNoir-YeastDerivativeA100ghL,PinotNoir,YeastDerivativeA100ghL,16,15
1,Merlot-Control,Merlot,Control,11,15
2,PinotNoir-YeastDerivativeB50ghL,PinotNoir,YeastDerivativeB50ghL,16,15
3,CabernetFrancNOBILE-P75gL,CabernetFrancNOBILE,P75gL,11,15
4,PinotNoir-Geosorb60ghL,PinotNoir,Geosorb60ghL,16,15
5,Merlot-Geosorb40ghL,Merlot,Geosorb40ghL,11,15
6,CSNOBILE-FreshGran24mo5gL,CSNOBILE,FreshGran24mo5gL,9,15
7,CabernetFranc-Geosorb100ghL,CabernetFranc,Geosorb100ghL,11,15
8,CabernetFranc-Polylact50ghL,CabernetFranc,Polylact50ghL,11,15
9,CabernetFranc-CaseiPlus50ghL,CabernetFranc,CaseiPlus50ghL,11,15


In [ ]:
survey_master_dataset

,survey_name,Date/Time,Attendee Code,Name,Email,Year Born,Years of Tasting Experience,Ethnicity,Country,Wine Region you work in,Gender,Aroma|Fruity,Aroma|Herbaceous,Aroma|Smoky,Aroma|Ashy,Aroma|Tar,Flavor|Fruity,Flavor|Herbaceous,Flavor|Smoky,Flavor|Ashy,Flavor|Tar,Mouthfeel|Astringent,Mouthfeel|Bitter,Mouthfeel|Round,Mouthfeel|Lingering Ash,Mouthfeel|Metallic,wine_type,treatment
5,PinotNoir-YeastDerivativeA100ghL,2021-01-21 09:09:48,JACOBINPIGEON,Daniel Dycus,daniel.dycus@laffort.com,1982,15,White,USA,USA,M,8,5,1,1,1,NaN,2,1,1,1,6,1,7,1,5,PinotNoir,YeastDerivativeA100ghL
6,PinotNoir-YeastDerivativeA100ghL,2021-01-21 09:27:32,FANCYPIGEON,Shaun Richardson,shaun.richardson@laffort.com,1968,30,Caucasian,USA,Sonoma,Male,4,2,4,4,5,4,2,4,6,5,3,3,5,6,5,PinotNoir,YeastDerivativeA100ghL
7,PinotNoir-YeastDerivativeA100ghL,2021-01-21 10:06:10,SPINIFEXPIGEON,Cass Gossage,cass.gossage@laffort.com,1979,14,caucasian,United States,Sonoma/Napa,male,7,2,2,2,1,7,1,2,3,1,2,1,6,5,1,PinotNoir,YeastDerivativeA100ghL
8,PinotNoir-YeastDerivativeA100ghL,2021-01-21 10:58:27,ICEPIGEON,Jillian Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,5,2,1,1,7,3,3,1,1,1,2,7,1,1,PinotNoir,YeastDerivativeA100ghL
9,PinotNoir-YeastDerivativeA100ghL,2021-01-21 11:01:28,RACINGPIGEON,Garett Savage,garett.savage@laffort.com,1978,22,Asian American,USA,NaN,NaN,6,4,2,1,1,6,3,2,1,1,4,1,6,2,2,PinotNoir,YeastDerivativeA100ghL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,CSNOBILE-Base10gL,2021-01-22 18:54:16,JACOBINPIGEON,Daniel Dycus,daniel.dycus@laffort.com,1982,15,White,USA,USA,M,5,3,8,7,8,4,2,9,NaN,NaN,5,8,3,NaN,9,CSNOBILE,Base10gL
10,CSNOBILE-Base10gL,2021-01-24 13:53:11,ARCHANGELPIGEON,Lindsey Otis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1,1,1,1,5,1,1,1,1,5,1,2,1,1,CSNOBILE,Base10gL
11,CSNOBILE-Base10gL,2021-01-24 15:42:03,BRONZEWINGPIGEON,Kelly Gifford,kelly.gifford@laffort.com,1988,NaN,Kennewick,United States,Pacific Northwest,Female,8,1,1,1,1,7,3,1,1,1,5,3,5,1,1,CSNOBILE,Base10gL
12,CSNOBILE-Base10gL,2021-01-25 14:19:15,HOMINGPIGEON,Matt Ridge,matt.ridge@laffort.com,1972,23,Irish America,USA,Western US,Male,4,5,1,1,1,3,5,1,1,1,1,1,5,1,1,CSNOBILE,Base10gL


In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'Wine Sensory'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
# include_index=False, include_column_header=True, resize=False
set_with_dataframe(sheet, survey_master_dataset) 